Importing Libraries

In [287]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, pandas as pd
import subprocess
import os


In [288]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Loading datasets

In [289]:
class LlmBaseline:
    def __init__(self, models=None, df=None):
        self.models = models if models else []
        self.dataframes = df if df else []
        self.summaries = []

    def add_df(self, df):
        self.dataframes.append(df)

    def read_dataset_path(self, path):
        self.dataframes.append(pd.read_csv(path))

    def get_all_df(self):
        return self.dataframes

    def generate_summary(self, model_name, prompt):
        # Send the prompt to model via Ollama (no model loaded in memory)
        result = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            text=True,
            capture_output=True
        )
        
        return result.stdout.strip()

    def generate_promt(self, data):
        prompt = f"""
            Based on the given data:
            {data}
            Your task is to write a brief, fluent,
            and coherent single-paragraph summary
            in natural language. The text should be
            balanced and neutral. Make sure that all the
            facts mentioned in the text can be derived
            from the input data, do *not* add any extra
            information.
        """
    
        return prompt

    # Convert the dataframe to readable formats i.e. csv and json
    def data_converter_csv(self, df):
        return df.to_csv(index=False)
    
    def data_converter_json(self, df):
        return df.reset_index().to_json(orient='records')

    def generator(self):
        if self.models == []:
            print("No models selected")
            return
        if self.dataframes == []:
            print("No dataframes added")
            return

        for df in self.dataframes:
            data = self.data_converter_json(df)
            prompt = self.generate_promt(data)
            summary = []
            
            for model in self.models:
                summary.append({"model": model, "summary": self.generate_summary(model, prompt)})

            self.summaries.append({"data": data, "summaries": summary})

        return self.summaries

    def save_summaries(self):
        if self.summaries == []:
            print("No summaries generated")
            return

        for i, summary in enumerate(summaries):
            for item in summary['summaries']:
                filename = f"summary_{i+1}_{item['model']}.txt"   # Creates file_1.txt, file_2.txt, etc.
                with open(filename, "w") as f:
                    f.write(item['summary'])
                print(f"Saved {filename}")

In [290]:
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

Current working directory: /Users/muhammadasad/Desktop/Data Mining/Project


In [291]:
# Load your dataset
df2010_2014 = pd.read_csv(current_directory + "/Datasets/2010_2014.csv")

df2015_2019 = pd.read_csv(current_directory + "/Datasets/2015_2019.csv")

df2020_2024 = pd.read_csv(current_directory + "/Datasets/2020_2024.csv")

In [292]:
df2020_2024.head()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),...,wind_direction_10m_dominant (°),winddirection_10m_dominant (°),wind_gusts_10m_mean (km/h),wind_speed_10m_mean (km/h),wind_gusts_10m_min (km/h),wind_speed_10m_min (km/h),relative_humidity_2m_mean (%),relative_humidity_2m_max (%),relative_humidity_2m_min (%),sunshine_duration (s)
0,2020-01-01,-1.4,1.0,-4.4,-6.8,-4.9,-8.7,0.2,0.0,0.14,...,252,252,31.7,16.6,14.4,6.9,67,78,50,9334.95
1,2020-01-02,3.6,8.3,-1.6,-1.1,2.6,-6.8,0.0,0.0,0.00,...,217,217,30.0,16.1,20.5,10.5,68,81,49,25522.97
2,2020-01-03,3.8,8.4,-0.7,0.5,4.6,-4.2,0.0,0.0,0.00,...,266,266,17.5,9.2,10.8,6.1,85,95,66,16050.96
3,2020-01-04,0.9,3.0,-0.4,-3.0,-1.2,-4.0,1.9,0.0,1.33,...,330,330,17.3,8.8,9.4,4.0,74,90,59,5190.75
4,2020-01-05,-0.2,2.0,-1.6,-4.8,-2.8,-6.5,2.8,0.0,1.96,...,266,266,24.0,12.3,15.1,8.0,71,91,56,7100.75


Data Preprocessing

In [293]:
df2010_2014.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [294]:
df2015_2019.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [295]:
df2020_2024.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [296]:
df2020_2024.dtypes

time                                object
temperature_2m_mean (°C)           float64
temperature_2m_max (°C)            float64
temperature_2m_min (°C)            float64
apparent_temperature_mean (°C)     float64
apparent_temperature_max (°C)      float64
apparent_temperature_min (°C)      float64
precipitation_sum (mm)             float64
rain_sum (mm)                      float64
snowfall_sum (cm)                  float64
precipitation_hours (h)            float64
wind_speed_10m_max (km/h)          float64
wind_gusts_10m_max (km/h)          float64
wind_direction_10m_dominant (°)      int64
winddirection_10m_dominant (°)       int64
wind_gusts_10m_mean (km/h)         float64
wind_speed_10m_mean (km/h)         float64
wind_gusts_10m_min (km/h)          float64
wind_speed_10m_min (km/h)          float64
relative_humidity_2m_mean (%)        int64
relative_humidity_2m_max (%)         int64
relative_humidity_2m_min (%)         int64
sunshine_duration (s)              float64
dtype: obje

In [297]:
# converting datatype of time column from object to datetime
df2010_2014['time'] = pd.to_datetime(df2010_2014['time'])
df2015_2019['time'] = pd.to_datetime(df2015_2019['time'])
df2020_2024['time'] = pd.to_datetime(df2020_2024['time'])

Calling the summary generator function

In [298]:
models = ["llama3", "gemma", "mistral"]
dataframes = [df2010_2014, df2015_2019, df2020_2024]

llm_baseline = LlmBaseline(models, dataframes)
summaries = llm_baseline.generator()
    

In [299]:
# print([(item['summaries']) for item in summaries])
summaries[0]['summaries']

[{'model': 'llama3',
  'summary': 'Here is a brief, fluent, and coherent single-paragraph summary of the given data:\n\nBetween January 1st, 2006 and February 2nd, 2007, the weather in this region experienced varying conditions. The temperature ranged from a low of -8.9°C to a high of 7.8°C, with an average mean temperature of 2.4°C. There were instances of precipitation, including snowfall on February 1st, and sunshine was recorded throughout the period. Wind speeds varied significantly, reaching up to 47.9 km/h, with gusts of up to 49.7 km/h. The relative humidity fluctuated between 34% and 96%, while the apparent temperature ranged from -14.7°C to 3.2°C. Overall, the weather during this period was marked by a mix of cold and warm temperatures, precipitation, and varying wind conditions.'},
 {'model': 'gemma',
  'summary': 'The provided dataset offers a comprehensive hourly record of weather conditions over a period of several days. Temperatures fluctuated widely, ranging from lows o

In [300]:
llm_baseline.save_summaries()

Saved summary_1_llama3.txt
Saved summary_1_gemma.txt
Saved summary_1_mistral.txt
Saved summary_2_llama3.txt
Saved summary_2_gemma.txt
Saved summary_2_mistral.txt
Saved summary_3_llama3.txt
Saved summary_3_gemma.txt
Saved summary_3_mistral.txt


In [308]:
df2010_2014.describe()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),...,wind_direction_10m_dominant (°),winddirection_10m_dominant (°),wind_gusts_10m_mean (km/h),wind_speed_10m_mean (km/h),wind_gusts_10m_min (km/h),wind_speed_10m_min (km/h),relative_humidity_2m_mean (%),relative_humidity_2m_max (%),relative_humidity_2m_min (%),sunshine_duration (s)
count,1826,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,...,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000
mean,2012-07-01 12:00:00.000000256,8.630887,12.619770,4.802848,6.084009,10.428642,1.891347,2.284940,1.958598,0.253510,...,220.290800,220.290800,26.666484,12.169496,14.622946,6.512432,71.179628,86.049288,54.726177,29942.863735
min,2010-01-01 00:00:00,-19.500000,-16.300000,-24.900000,-27.000000,-22.700000,-30.300000,0.000000,0.000000,0.000000,...,0.000000,0.000000,9.200000,3.800000,2.900000,0.000000,33.000000,49.000000,22.000000,0.000000
25%,2011-04-02 06:00:00,0.300000,3.400000,-2.400000,-4.300000,-0.900000,-7.000000,0.000000,0.000000,0.000000,...,147.500000,147.500000,19.700000,8.700000,9.400000,3.400000,63.000000,80.000000,44.000000,19921.472500
50%,2012-07-01 12:00:00,8.900000,12.800000,5.150000,5.600000,9.950000,1.400000,0.000000,0.000000,0.000000,...,244.000000,244.000000,25.100000,11.400000,12.600000,6.000000,71.000000,88.000000,54.000000,32400.000000
75%,2013-09-30 18:00:00,17.900000,22.200000,13.800000,17.500000,22.200000,12.700000,2.000000,1.200000,0.000000,...,296.000000,296.000000,32.300000,14.900000,18.400000,8.975000,79.000000,94.000000,64.000000,42858.740000
max,2014-12-31 00:00:00,30.500000,34.800000,25.800000,33.200000,38.000000,29.000000,48.800000,48.800000,20.020000,...,360.000000,360.000000,73.600000,33.100000,52.200000,22.000000,98.000000,100.000000,96.000000,51718.020000
std,NaN,10.599886,10.971047,10.418750,12.922165,13.434982,12.538370,5.034955,4.816576,1.145414,...,96.126306,96.126306,9.236191,4.562563,7.270312,4.051253,11.125069,9.690033,13.501925,15447.939452
